In [141]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [142]:
def get_df(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file_path} does not exist.")
    if not file_path.endswith('.csv'):
        raise ValueError(f"File {file_path} is not a CSV file.")
    try:
        df = pd.read_csv(file_path)
    except pd.errors.EmptyDataError:
        raise ValueError(f"File {file_path} is empty.")
    except pd.errors.ParserError:
        raise ValueError(f"File {file_path} is corrupted or not a valid CSV.")
    if df.empty:
        raise ValueError(f"DataFrame from {file_path} is empty.")
    return df

In [143]:
# df에서 'file' 열을 기준으로 그룹화
def group_by_file(df):
    if 'file' not in df.columns:
        raise ValueError("'file' column is missing in the DataFrame.")
    grouped = df.groupby('file')
    return grouped

In [144]:
# group 내에서 각 metric에 대해 최대값을 찾고, 해당하는 row의 'HP' 값을 반환
def get_max_hp(group, metric):
    if metric not in group.columns:
        raise ValueError(f"'{metric}' column is missing in the group.")
    max_row = group.loc[group[metric].idxmax()]
    return max_row['HP']

In [145]:
# 위 주석 처리 코드를 함수로 변경
def find_max_hp_per_metric(grouped_df, metrics):
    max_hps = {}
    for metric in metrics:
        max_hps[metric] = []
        for file_name, group in grouped_df:
            try:
                max_hp = get_max_hp(group, metric)
                max_hps[metric].append(max_hp)
                #print(f'{file_name} - {metric}: {max_hp}')
            except ValueError as e:
                print(f"Error processing {file_name}: {e}")

    # 가장 중복이 많은 max_hp를 찾기
    max_hp_counts = {}
    for metric, hps in max_hps.items():
        hp_counts = pd.Series(hps).value_counts()
        max_hp = hp_counts.idxmax()
        count = hp_counts.max()
        max_hp_counts[metric] = (max_hp, count)
        #print(f"Metric: {metric}, Max HP: {max_hp}, Count: {count}")
    
    return max_hp_counts


In [146]:
metrics = ['AUC-PR','AUC-ROC','VUS-PR','VUS-ROC','Standard-F1','PA-F1','Event-based-F1','R-based-F1','Affiliation-F']

In [7]:
df = get_df('/home/hwkang/dev-TSB-AD/TSB-AD/eval/metrics/multi/SpikeCNN_proposed.csv')
grouped_df = group_by_file(df)
best_hps = find_max_hp_per_metric(grouped_df, metrics)

# best_hps를 반복문으로 돌면서 각 metric에 대해 max_hp와 count를 출력
for metric, (max_hp, count) in best_hps.items():
    print(f"Metric: {metric}, Max HP: {max_hp}, Count: {count}")

# best_hps for SpikeCNN_proposed
# 1. {'window_size': 100, 'num_channel': [32, 64], 'num_encoded_features': 16}: 4개 지표에서 최다
# 2. {'window_size': 50, 'num_channel': [32, 40], 'num_encoded_features': 4}: 4개 지표에서 최다

In [ ]:
df = get_df('/home/hwkang/dev-TSB-AD/TSB-AD/eval/metrics/multi/SpikeCNN_conv.csv')
grouped_df = group_by_file(df)
best_hps = find_max_hp_per_metric(grouped_df, metrics)

# best_hps를 반복문으로 돌면서 각 metric에 대해 max_hp와 count를 출력
for metric, (max_hp, count) in best_hps.items():
    print(f"Metric: {metric}, Max HP: {max_hp}, Count: {count}")

# best_hps for SpikeCNN_conv
# 1. {'window_size': 50, 'num_channel': [32, 64], 'num_encoded_features': 16}

In [ ]:
df = get_df('/home/hwkang/dev-TSB-AD/TSB-AD/eval/metrics/multi/SpikeCNN_delta.csv')
grouped_df = group_by_file(df)
best_hps = find_max_hp_per_metric(grouped_df, metrics)

# best_hps를 반복문으로 돌면서 각 metric에 대해 max_hp와 count를 출력
for metric, (max_hp, count) in best_hps.items():
    print(f"Metric: {metric}, Max HP: {max_hp}, Count: {count}")

# best_hps for SpikeCNN_proposed
# 1. {'window_size': 100, 'num_channel': [32, 64], 'num_encoded_features': 16}

In [ ]:
df = get_df('/home/hwkang/dev-TSB-AD/TSB-AD/eval/tuning/multi/Proposal_tuning_conv.csv')
grouped_df = group_by_file(df)
best_hps = find_max_hp_per_metric(grouped_df, metrics)

# best_hps를 반복문으로 돌면서 각 metric에 대해 max_hp와 count를 출력
for metric, (max_hp, count) in best_hps.items():
    print(f"Metric: {metric}, Max HP: {max_hp}, Count: {count}")

# best_hps for Proposal_tuning_conv from VUS-PR
# 1. {'num_enc_features': 4, 'spike_mode': 'TD', 'surrogate_mode': 'atan'}

Metric: AUC-PR, Max HP: {'num_enc_features': 4, 'spike_mode': 'TD', 'surrogate_mode': 'atan'}, Count: 3
Metric: AUC-ROC, Max HP: {'num_enc_features': 8, 'spike_mode': 'TRB', 'surrogate_mode': 'atan'}, Count: 2
Metric: VUS-PR, Max HP: {'num_enc_features': 4, 'spike_mode': 'TD', 'surrogate_mode': 'atan'}, Count: 4
Metric: VUS-ROC, Max HP: {'num_enc_features': 4, 'spike_mode': 'TD', 'surrogate_mode': 'atan'}, Count: 2
Metric: Standard-F1, Max HP: {'num_enc_features': 4, 'spike_mode': 'GS', 'surrogate_mode': 'fastsig'}, Count: 4
Metric: PA-F1, Max HP: {'num_enc_features': 4, 'spike_mode': 'GS', 'surrogate_mode': 'fastsig'}, Count: 8
Metric: Event-based-F1, Max HP: {'num_enc_features': 4, 'spike_mode': 'GS', 'surrogate_mode': 'fastsig'}, Count: 7
Metric: R-based-F1, Max HP: {'num_enc_features': 4, 'spike_mode': 'GS', 'surrogate_mode': 'fastsig'}, Count: 5
Metric: Affiliation-F, Max HP: {'num_enc_features': 4, 'spike_mode': 'GS', 'surrogate_mode': 'fastsig'}, Count: 7


In [ ]:
df = get_df('/home/hwkang/dev-TSB-AD/TSB-AD/eval/tuning/multi/SpikeCNN_tuning_conv.csv')
grouped_df = group_by_file(df)
best_hps = find_max_hp_per_metric(grouped_df, metrics)

# best_hps를 반복문으로 돌면서 각 metric에 대해 max_hp와 count를 출력
for metric, (max_hp, count) in best_hps.items():
    print(f"Metric: {metric}, Max HP: {max_hp}, Count: {count}")

# best_hps for SpikeCNN_tuning_conv from VUS-PR
# 1. {'window_size': 100, 'num_channel': [32, 40], 'num_enc_features': 16}

Metric: AUC-PR, Max HP: {'window_size': 50, 'num_channel': [32, 40], 'num_enc_features': 8}, Count: 3
Metric: AUC-ROC, Max HP: {'window_size': 150, 'num_channel': [32, 64], 'num_enc_features': 8}, Count: 4
Metric: VUS-PR, Max HP: {'window_size': 100, 'num_channel': [32, 40], 'num_enc_features': 16}, Count: 3
Metric: VUS-ROC, Max HP: {'window_size': 150, 'num_channel': [32, 40], 'num_enc_features': 16}, Count: 3
Metric: Standard-F1, Max HP: {'window_size': 50, 'num_channel': [32, 40], 'num_enc_features': 4}, Count: 5
Metric: PA-F1, Max HP: {'window_size': 50, 'num_channel': [32, 40], 'num_enc_features': 4}, Count: 10
Metric: Event-based-F1, Max HP: {'window_size': 50, 'num_channel': [32, 40], 'num_enc_features': 4}, Count: 8
Metric: R-based-F1, Max HP: {'window_size': 50, 'num_channel': [32, 40], 'num_enc_features': 4}, Count: 4
Metric: Affiliation-F, Max HP: {'window_size': 50, 'num_channel': [32, 40], 'num_enc_features': 4}, Count: 7
